In [7]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_empirical_fixed_cost_runs(acq, tfn):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOPT", filters={
        "sweep": "nxb6ejc0", 
        "config.problem": tfn,
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [6]:
acquisition_functions = {
    'RandomSearch': 'RandomSearch',
    'ExpectedImprovement':'ExpectedImprovement',
    'ThompsonSampling':'ThompsonSampling', 
    'KnowledgeGradient':'KnowledgeGradient',
    'MultiStepLookaheadEI':'MultiStepLookaheadEI',
    'Gittins_Lambda_01':'Gittins_Lambda01',
    'Gittins_Lambda_001':'Gittins_Lambda001',
    'Gittins_Lambda_0001':'Gittins_Lambda0001',
    'Gittins_Step_Divide2':'Gittins_Step_Divide2',
    'Gittins_Step_Divide5':'Gittins_Step_Divide5',
    'Gittins_Step_Divide10':'Gittins_Step_Divide10',
    'Gittins_Step_EIpu':'Gittins_Step_EIPC',
    }
target_functions = ["PestControl"]

In [8]:
grouped_runs = {(a,t): load_empirical_fixed_cost_runs(a,t) for a in acquisition_functions.keys() for t in target_functions}

100%|██████████| 16/16 [00:08<00:00,  1.98it/s]


In [9]:
for a in acquisition_functions.keys():
    for t in target_functions:
        config_and_metrics_per_seed = grouped_runs[a,t]

        cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed]).T
        best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed]).T

        print(a, t, best_observed_per_seed.shape)
        
        best_25 = np.quantile(best_observed_per_seed, 0.25, axis=1)
        best_50 = np.quantile(best_observed_per_seed, 0.5, axis=1)
        best_75 = np.quantile(best_observed_per_seed, 0.75, axis=1)

        output = np.stack((cumulative_cost_per_seed.mean(axis=1), best_25, best_50, best_75),axis=-1)

        np.savetxt(f"results/preprint/empirical/Empirical_{t}_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

RandomSearch PestControl (101, 16)
ExpectedImprovement PestControl (101, 16)
ThompsonSampling PestControl (101, 16)
KnowledgeGradient PestControl (101, 16)
MultiStepLookaheadEI PestControl (101, 16)
Gittins_Lambda_01 PestControl (101, 16)
Gittins_Lambda_001 PestControl (101, 16)
Gittins_Lambda_0001 PestControl (101, 16)
Gittins_Step_Divide2 PestControl (101, 16)
Gittins_Step_Divide5 PestControl (101, 16)
Gittins_Step_Divide10 PestControl (101, 16)
Gittins_Step_EIpu PestControl (101, 16)
